# GenAI Website Safety Analyzer


In [1]:
!pip3 install openai beautifulsoup4 requests validators

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import openai
import os


os.environ["OPENAI_API_KEY"] = "API nhi bataunga"


openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = "https://api.groq.com/openai/v1"  
model_name = "llama3-70b-8192"  

In [3]:

def analyze_with_ai(text_content):
    prompt = f"""
You are a cybersecurity assistant. Analyze the following website content and tell if it is potentially harmful, scammy, misleading, or safe. Give your result as:
- Safety Status: Safe / Caution / Dangerous
- Reason:
- Suggestions:

Content:
{text_content[:3000]}
"""
    try:
        response = openai.ChatCompletion.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=300,
            temperature=0.2,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Groq API Error: {str(e)}"

In [4]:

import requests
from bs4 import BeautifulSoup
import validators

def analyze_website(url):
    if not validators.url(url):
        return {"error": "Invalid URL"}

    domain = url.split('//')[-1].split('/')[0]
    is_https = url.startswith("https://")

    try:
        res = requests.get(url, timeout=5)
        soup = BeautifulSoup(res.text, 'html.parser')

        title = soup.title.string.strip() if soup.title else "No Title Found"
        desc = soup.find("meta", attrs={"name": "description"})
        description = desc["content"] if desc else "No Description"

        icon_tag = soup.find("link", rel=lambda x: x and 'icon' in x.lower())
        favicon = icon_tag.get("href", "") if icon_tag else "Not Found"
        if favicon.startswith("/"):
            favicon = url.rstrip("/") + favicon

        text_content = soup.get_text(separator=' ', strip=True)
        ai_review = analyze_with_ai(text_content)

        return {
            "url": url,
            "domain": domain,
            "is_https": is_https,
            "title": title,
            "description": description,
            "favicon": favicon,
            "ai_review": ai_review,
            "status": "Groq-reviewed"
        }
    except Exception as e:
        return {"error": str(e)}

In [5]:

result = analyze_website("https://www.wikipedia.org")
from pprint import pprint
pprint(result)

{'ai_review': '**Safety Status: Safe**\n'
              '\n'
              '**Reason:** The provided content appears to be a legitimate '
              'Wikipedia page, listing various languages and their '
              'corresponding article counts. The content is informative, '
              'neutral, and does not contain any suspicious links, phishing '
              'attempts, or malicious code.\n'
              '\n'
              '**Suggestions:**\n'
              '\n'
              '* Verify the authenticity of the website by checking the URL '
              'and ensuring it starts with "https" and "wikipedia.org".\n'
              '* Be cautious when clicking on links from unfamiliar sources, '
              'even if they appear to be from Wikipedia.\n'
              '* Keep your browser and operating system up to date to ensure '
              'you have the latest security patches and protections.\n'
              "* If you're unsure about the legitimacy of a website or "
    